In [2]:
import pandas as pd

In [5]:
%matplotlib inline

In [16]:
num = 750

In [17]:
ratings = pd.read_csv('load_test_db/{}/Rating.csv'.format(num),header=None)
users = pd.read_csv('load_test_db/{}/Users.csv'.format(num),header=None)
v2s = pd.read_csv('load_test_db/{}/VideoRoadSeg.csv'.format(num),header=None)
seg = pd.read_csv('load_test_db/{}/RoadSegment.csv'.format(num),header=None)
login = pd.read_csv('load_test_db/{}/loginLog.csv'.format(num),header=None)

In [31]:
ratings.columns = ['rid', 'uid','email','vid','rating','comment','tags','familiar','time','tz',10,'ytlog','lid']
v2s.columns = ['vrsid','vid','sid','clip_name','index_seg','ratio',]
seg.columns=['sid','segmentid','index_seg','sumScore','sumRatio','sumCnt','geometry',]

In [32]:
users_in_ratings = ratings.groupby('uid').nunique()

In [33]:
# unique vids per user 
users_in_ratings['vid'].value_counts()

7    750
Name: vid, dtype: int64

In [24]:
login[1].nunique()==num, len(login)==num, users[1].nunique()==num

(True, True, True)

In [41]:
r_on_v = ratings.groupby('vid')['rating'].agg(['sum','count']).reset_index()

In [47]:
s_from_r = r_on_v.merge(v2s)[['sid', 'count', 'sum', 'ratio']]

In [63]:
s_from_r['total_ratio'] = s_from_r['count'] * s_from_r.ratio
s_from_r['total_sum'] = s_from_r['sum'] * s_from_r.ratio

In [65]:
s2s = s_from_r.groupby('sid')[['total_sum','count', 'total_ratio']].sum().reset_index().merge(seg)

In [68]:
s2s

,sid,total_sum,count,total_ratio,segmentid,index_seg,sumScore,sumRatio,sumCnt,geometry
0,1,3905.0,2117,1058.5,1,1,3905.00,1058.500,2117,LINESTRING (-77.01364744974134 38.956284074045...
1,2,8762.4,4345,2379.6,2,2,8762.47,2379.550,4345,LINESTRING (-77.01357027112063 38.954497383391...
2,3,8694.4,4287,2343.6,3,3,8694.44,2343.700,4287,LINESTRING (-77.01362123887805 38.955569541454...
3,4,15630.4,6386,4236.8,4,4,15630.40,4236.800,6386,LINESTRING (-77.01324339541158 38.951215745942...
4,5,2338.8,2115,634.5,5,5,2338.79,634.486,2115,LINESTRING (-77.01346048293019 38.952283884461...
